In [10]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re 

#Importamos librerías necesarias para la visualización
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [11]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode



In [12]:
df_HR = pd.read_csv('df_HR_limpio.csv')
df_HR.head()

,Unnamed: 0,Age,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RemoteWork
0,0,51,No,Unknown,684.0,6,3,Unknown,"162,0",1,M,51.0,3,5,Research Director,3,Unknown,19537.0,6462,7,No,13,"3,0",3,0,Unknown,5,3.000000,20,15,15,1972,Yes
1,1,52,No,Unknown,699.0,1,4,Life Sciences,"259,0",3,M,65.0,2,5,Manager,3,Unknown,19999.0,5678,0,Unknown,14,"3,0",1,1,"34,0",5,3.000000,33,11,9,1971,1
2,2,42,No,travel_rarely,532.0,4,2,Technical Degree,"319,0",3,M,58.0,3,5,Manager,4,Married,19232.0,4933,1,No,11,"3,0",4,0,"22,0",3,2.766568,22,11,15,1981,1
3,3,47,No,travel_rarely,359.0,2,4,Medical,Unknown,1,F,82.0,3,4,Research Director,3,Married,17169.0,26703,3,Unknown,19,"3,0",2,2,Unknown,2,2.766568,20,5,6,1976,False
4,5,48,No,Unknown,117.0,22,3,Medical,"1900,0",4,F,58.0,3,4,Manager,4,Unknown,17174.0,2437,3,No,11,"3,0",2,1,Unknown,3,3.000000,22,4,7,1975,Yes


In [13]:
df_HR.shape

(1080, 33)

In [14]:
df_Employees = df_HR[["Age", "Gender", "MaritalStatus", "DateBirth", "Education", "EducationField", "Attrition", "BusinessTravel", "DistanceFromHome"]]

df_JobDetails = df_HR[["JobRole", "JobLevel", "NumCompaniesWorked", "TotalWorkingYears", "YearsAtCompany", "YearsSinceLastPromotion", "YearsWithCurrManager"]]

df_Compensation = df_HR[["DailyRate", "HourlyRate", "MonthlyIncome", "MonthlyRate", "StockOptionLevel", "PercentSalaryHike", "OverTime", "TrainingTimesLastYear", "RemoteWork"]]

df_Satisfaction = df_HR[["EnvironmentSatisfaction", "JobInvolvement", "JobSatisfaction", "RelationshipSatisfaction", "WorkLifeBalance", "PerformanceRating"]]

In [15]:
df_empleados = pd.read_csv('Employees.csv')
df_empleados.shape

(1080, 9)

df_employees a lista de tuplas

In [16]:
lista_de_tuplas = [tuple(x) for x in df_Employees.to_records(index=False)] 

In [17]:
lista_de_tuplas

[(51, 'M', 'Unknown', 1972, 3, 'Unknown', 'No', 'Unknown', 6),
 (52, 'M', 'Unknown', 1971, 4, 'Life Sciences', 'No', 'Unknown', 1),
 (42, 'M', 'Married', 1981, 2, 'Technical Degree', 'No', 'travel_rarely', 4),
 (47, 'F', 'Married', 1976, 4, 'Medical', 'No', 'travel_rarely', 2),
 (48, 'F', 'Unknown', 1975, 3, 'Medical', 'No', 'Unknown', 22),
 (59, 'F', 'Unknown', 1964, 3, 'Life Sciences', 'No', 'Unknown', 25),
 (42, 'M', 'Married', 1981, 1, 'Unknown', 'No', 'travel_rarely', 1),
 (41, 'M', 'Unknown', 1982, 3, 'Unknown', 'No', 'travel_frequently', 9),
 (41, 'F', 'Unknown', 1982, 1, 'Unknown', 'No', 'non-travel', 7),
 (41, 'F', 'Married', 1982, 3, 'Unknown', 'Yes', 'travel_rarely', 12),
 (40, 'F', 'Unknown', 1983, 4, 'Unknown', 'No', 'travel_rarely', 2),
 (58, 'M', 'Single', 1965, 4, 'Unknown', 'Yes', 'travel_rarely', 2),
 (45, 'M', 'Unknown', 1978, 1, 'Unknown', 'No', 'non-travel', 1),
 (33, 'F', 'Single', 1990, 2, 'Life Sciences', 'No', 'Unknown', 1),
 (47, 'F', 'Married', 1976, 2, 'Othe

In [18]:
df_Employees.shape

(1080, 9)

Crear base de datos

In [19]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE ABC_Corporation")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: CREATE DATABASE ABC_Corporation


Creacion de tablas

In [20]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Employees (
        EmployeeNumber INT PRIMARY KEY AUTO_INCREMENT,
        Age INT,
        Gender VARCHAR(100),
        MaritalStatus VARCHAR(100),
        DateBirth INT,
        Education INT,
        EducationField VARCHAR(100),
        Attrition VARCHAR(100),
        BusinessTravel VARCHAR(100),
        DistanceFromHome INT
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: 
        CREATE TABLE Employees (
      ..


Insercion de datos

In [21]:
cnx = mysql.connector.connect(user = 'root',
                              password = 'AlumnaAdalab',
                              host = '127.0.0.1',
                              database = 'ABC_Corporation'
                              )


mycursor = cnx.cursor()
sql = """INSERT INTO Employees (
            Age, Gender, MaritalStatus, DateBirth, Education, EducationField, Attrition, BusinessTravel, DistanceFromHome
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
val = [
]

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

-1 registro/s insertado/s.


### extraer en lista de tuplas Job Details

In [22]:
df_job_details = pd.read_csv('JobDetails.csv')
df_job_details.shape

(1080, 7)

In [23]:
job_details = [list(x) for x in df_job_details .to_records(index=False)] 

In [24]:
# ver que coincidan los registros de las tuplas con el df
job_details

[[5, ' Research Director ', 7, 'Unknown', 20, 15, 15],
 [5, ' Manager ', 0, '34,0', 33, 11, 9],
 [5, ' Manager ', 1, '22,0', 22, 11, 15],
 [4, ' Research Director ', 3, 'Unknown', 20, 5, 6],
 [4, ' Manager ', 3, 'Unknown', 22, 4, 7],
 [3, ' Sales Executive ', 7, '28,0', 21, 7, 9],
 [2, ' Sales Executive ', 1, '20,0', 20, 11, 6],
 [5, ' Research Director ', 2, '21,0', 18, 0, 11],
 [5, ' Research Director ', 1, 'Unknown', 21, 5, 10],
 [5, ' Research Director ', 1, 'Unknown', 22, 15, 8],
 [5, ' Manager ', 0, 'Unknown', 20, 1, 12],
 [5, ' Research Director ', 7, '40,0', 31, 13, 8],
 [3, ' Healthcare Representative ', 3, '25,0', 23, 14, 4],
 [3, ' Research Director ', 1, '15,0', 15, 8, 12],
 [4, ' Manager ', 1, 'Unknown', 26, 3, 0],
 [2, ' Manufacturing Director ', 9, '18,0', 16, 5, 12],
 [2, ' Sales Executive ', 1, '15,0', 15, 0, 7],
 [3, ' Sales Executive ', 0, 'Unknown', 22, 13, 5],
 [3, ' Healthcare Representative ', 2, '25,0', 17, 12, 11],
 [5, ' Manager ', 4, '26,0', 14, 4, 8],
 [2, '

In [25]:
tuplas_job_details

NameError: name 'tuplas_job_details' is not defined

### Crear tabla JobDetails

In [26]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE JobDetails (
        JobDetailID INT PRIMARY KEY AUTO_INCREMENT,
        JobRole VARCHAR(100),
        JobLevel INT,
        NumCompaniesWorked INT,
        TotalWorkingYears VARCHAR(100),
        YearsAtCompany INT,
        YearsSinceLastPromotion INT,
        YearsWithCurrManager INT,
        EmployeeNumber INT,
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: 
        CREATE TABLE JobDetails (
     ..


### Crear tabla Compensation

In [27]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Compensation (
        CompensationID INT PRIMARY KEY AUTO_INCREMENT,
        EmployeeNumber INT,
        DailyRate INT,
        HourlyRate INT,
        MonthlyIncome INT,
        MonthlyRate INT,
        StockOptionLevel INT,
        PercentSalaryHike INT,
        OverTime VARCHAR(100),
        TrainingTimesLastYear INT,
        RemoteWork VARCHAR(100),
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: 
        CREATE TABLE Compensation (
   ..


### Creacion tabla Satisfaction

In [28]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Satisfaction (
        SatisfactionID INT PRIMARY KEY AUTO_INCREMENT,
        EmployeeNumber INT,
        EnvironmentSatisfaction INT,
        JobInvolvement INT,
        JobSatisfaction INT,
        RelationshipSatisfaction INT,
        WorkLifeBalance INT,
        PerformanceRating VARCHAR(100),
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: 
        CREATE TABLE Satisfaction (
   ..
